<a href="https://colab.research.google.com/github/Mesh-Sys/vui/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [3]:
!conda init --all

no change     /usr/local/condabin/conda
no change     /usr/local/bin/conda
no change     /usr/local/bin/conda-env
no change     /usr/local/bin/activate
no change     /usr/local/bin/deactivate
no change     /usr/local/etc/profile.d/conda.sh
no change     /usr/local/etc/fish/conf.d/conda.fish
no change     /usr/local/shell/condabin/Conda.psm1
no change     /usr/local/shell/condabin/conda-hook.ps1
no change     /usr/local/lib/python3.11/site-packages/xontrib/conda.xsh
no change     /usr/local/etc/profile.d/conda.csh
modified      /root/.bashrc
modified      /root/.zshrc
modified      /root/.config/fish/config.fish
modified      /root/.xonshrc
modified      /root/.tcshrc

==> For changes to take effect, close and re-open your current shell. <==



In [2]:
!conda create -n vui python==3.12.3

Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/vui

  added / updated specs:
    - python==3.12.3


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.7.9   |       hbd8a1cb_0         149 KB  conda-forge
    icu-75.1                   |       he02047a_0        11.6 MB  conda-forge
    ld_impl_linux-64-2.44      |       h1423503_1         660 KB  conda-forge
    libexpat-2.7.0             |       h5888daf_0          73 KB  conda-forge
    libffi-3.4.6               |       h2dba641_1          56 KB  conda-forge
    libgcc-15.1.0              |       h767d61c_3         806 KB  conda-forge
    l

In [5]:
!mv $(which python) $(which python).bak
!mv $(which python3) $(which python3).bak
!mv $(which pip) $(which pip).bak
!mv $(which pip3) $(which pip3).bak
!ln -s /usr/local/envs/vui/bin/python /usr/local/bin/python
!ln -s /usr/local/envs/vui/bin/python3 /usr/local/bin/python3
!ln -s /usr/local/envs/vui/bin/pip /usr/local/bin/pip
!ln -s /usr/local/envs/vui/bin/pip3 /usr/local/bin/pip3

Restart session

In [ ]:
!git clone https://github.com/Mesh-Sys/vui.git
%cd vui

In [ ]:
import torch
import time
import torchaudio

from vui.model import Vui
from vui.notebook import play
from vui.fluac import Fluac
from vui.inference import render as generate

model = Vui.from_pretrained_inf(Vui.ABRAHAM).cuda()
model.decoder = torch.compile(model.decoder, fullgraph=True, mode="reduce-overhead")

In [ ]:
short_samples = ["""Welcome to Fluxions, the podcast where... we uh explore how technology is shaping the world around us. I'm your host, Alex.
[breath] And I'm Jamie um [laugh] today, we’re diving into a [hesitate] topic that’s transforming customer service uh voice technology for agents.
That’s right. We’re [hesitate] talking about the AI-driven tools that are making those long, frustrating customer service calls a little more bearable, for both the customer and the agents.
"""]

radio = """Um, hey Sarah, so I just left the meeting with the, uh, rabbit focus group and they are absolutely loving the new heritage carrots! Like, I've never seen such enthusiastic thumping in my life! The purple ones are testing through the roof - apparently the flavor profile is just amazing - and they're willing to pay a premium for them! We need to, like, triple production on those immediately and maybe consider a subscription model? Anyway, gotta go, but let's touch base tomorrow about scaling this before the Easter rush hits!"""
short_samples.append(radio)
short_samples.append("""What an absolute joke, like I'm really not enjoying this situation where I'm just forced to say things.""")
short_samples.append(""" So [breath] I don't know if you've been there [breath] but I'm really pissed off.
Oh no! Why, what happened?
Well I went to this cafe hearth, and they gave me the worst toastie I've ever had, it didn't come with salad it was just raw.
Well that's awful what kind of toastie was it?
It was supposed to be a chicken bacon lettuce tomatoe, but it was fucking shite, like really bad and I honestly would have preferred to eat my own shit.
[laugh] well, it must have been awful for you, I'm sorry to hear that, why don't we move on to brighter topics, like the good old weather?
""")

temperature = .5
top_p = None
top_k = 100
short_samples.reverse()
    sr = model.codec.config.sample_rate

for chunk in short_samples:
    text=f"{chunk.strip()}"
    print(text)

    t1 = time.perf_counter()
    waveform = generate(
        model,
           text,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
    )
    print(waveform.shape[-1]/(sr*(time.perf_counter()-t1)), "x")
    print(waveform.shape)
    print("total time", time.perf_counter() - t1)

    sample_rate = 22050
    waveform = waveform[...,: -4000]
    play(waveform.flatten().cpu(), sr, False)
    # out_path = f"{temperature=}-{top_p=}-{top_k=}-{guidance_scale=}-{int(time.time())}.mp3"
    # print(f"Saved to {out_path}")
    # torchaudio.save(out_path, waveform, sample_rate)